In [3]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import time
import random
import os
import logging
from datetime import datetime
import os
import sys
import requests
import pandas as pd
import re

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_11076\1378756733.py:13: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [10]:
udemy_cat_tag_df = pd.read_csv("../data-dump/udemy_lecture_cat_tags.csv")
udemy_cat_tag_df

,id,isAccess,category,tags
0,5467400,True,자격증,Professional Scrum Master(PSM)
1,1294046,True,모바일 개발,Android 개발
2,3620656,True,웹 개발,JavaScript
3,3825092,True,기타,WordPress
4,3843498,True,자격증,Tableau
...,...,...,...,...
59550,4693788,True,보안/네트워크,Powershell
59551,352378,True,기타,Python
59552,5510828,True,기타,JavaScript
59553,3381284,True,보안/네트워크,사이버 보안


In [4]:
def apply_logic(html_file, json_file):
    # 이 곳에 로직을 적용하세요.
    pass

# 폴더 경로
folder_path = "./page/" # 실제 폴더 경로로 변경해주세요.

html_files = {f[:-5] for f in os.listdir(folder_path) if f.endswith('.html')}
json_files = {f[:-5] for f in os.listdir(folder_path) if f.endswith('.json')}


In [2]:
def open_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)
def open_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        contents = f.read()
    return BeautifulSoup(contents, 'html.parser')

In [5]:
str2level = {
    "초급자" : "BEGINNER",
    "중급자" : "INTERMEDIATE",
    "전문가" : "EXPERT",
    "모든 수준" : "ALL",
}
currency2won = {
    "CAD" : 996.94,
    "USD" : 1339.35
}

In [6]:
json_list = open_json("./list/parted/list.json")

In [44]:
def get_lecture_dic(pk):
    html_file = os.path.join(folder_path, f"{pk}.html")
    json_file = os.path.join(folder_path, f"{pk}.json")
    json_description = open_json(json_file)
    json_info = json_list.get(pk)
    html_soup = open_html(html_file)
    
    lecture = {}

    sections = json_description["curriculum_context"]["data"]["sections"]
    total_time = 0
    for section in sections:
        total_time += section["content_length"]

    what_you_learn = list(map(lambda x:x.text, html_soup.find_all(class_="objective--objective-item--AHk7S")))


    curriculum = {}
    curriculum["curriculum"] = []
    for section in sections:
        temp = {}
        temp["time"] = int(section["content_length"]//60)
        temp["item_count"] = section["lecture_count"]
        temp["title"] = section["title"]
        items = []
        for item in section["items"]:
            items.append({
                "title" : item["title"],
                "time" : item["content_summary"]
            })
        temp["items"] = items
        curriculum["curriculum"].append(temp)

    price_dic = json_description["price_text"]["data"]["pricing_result"]["price"]
    sale_price_dic = json_description["price_text"]["data"]["pricing_result"]["list_price"]
    # int(currency2won.get(price_dic["currency"])*price_dic["amount"])//5000*5000
    # int(currency2won.get(sale_price_dic["currency"])*sale_price_dic["amount"])//5000*5000

    # lecture["lecture_id"] = 1
    lecture["site_lecture_id"] = f"u{json_info["id"]}"
    lecture["name"] = json_info["title"]
    lecture["image"] = json_info["image_750x422"]
    lecture["category_name"] = udemy_cat_tag_df[udemy_cat_tag_df["id"]==json_info["id"]]["category"].item()
    # 을 넣어놓으면 조인해서 id받아올수있다. 
    # lecture["category_id"]
    lecture["price_original"] = int(currency2won.get(price_dic["currency"])*price_dic["amount"])//5000*5000
    lecture["price_sale"] = int(currency2won.get(sale_price_dic["currency"])*sale_price_dic["amount"])//5000*5000
    lecture["level"] = str2level[json_info["instructional_level_simple"]]
    lecture["summary"] = "||".join(list(map(lambda x:x, what_you_learn)))
    lecture["description_summary"] = json_info["headline"]
    # lecture["description_detail"] html이라 일단 더미
    lecture["site_review_rating"] = json_description["slider_menu"]["data"]["rating"]
    lecture["site_review_count"] = json_description["slider_menu"]["data"]["num_reviews"]
    lecture["site_student_count"] = json_description["slider_menu"]["data"]["num_students"]
    lecture["review_sum"] = 0
    lecture["review_count"] = 0
    lecture["total_time"] = int(total_time//60)
    lecture["curriculum"] = curriculum
    lecture["instructor"] = json_info["visible_instructors"][0]["title"]
    lecture["site_type"] = "UDEMY"
    lecture["site_link"] = f"https://udemy.com{json_info["url"]}"
    # lecture["gpt_review_good"] = 
    # lecture["gpt_review_bad"] = 
    # lecture["create_time"] = 
    # lecture["update_time"] = 

    return lecture

In [8]:
udemy_lectures = []
times = set()
for pk in tqdm((html_files & json_files)):
    try:
        json_file = os.path.join(folder_path, f"{pk}.json")
        json_description = open_json(json_file)
        json_info = json_list.get(pk)
        
        curriculum = {}
        curriculum["curriculum"] = []
        sections = json_description["curriculum_context"]["data"]["sections"]

        for section in sections:
            for item in section["items"]:
                times.add(item["content_summary"])
    except:
        continue


100%|██████████| 60130/60130 [00:59<00:00, 1006.91it/s]


In [9]:
times

{'',
 '01:52:05',
 '02:50',
 '15:55',
 '46:53',
 '1 trang',
 '11 întrebări',
 '02:50:37',
 '01:25:35',
 '01:05:11',
 '01:28:10',
 '200 questions',
 '31 pertanyaan',
 '06:43',
 '48:40',
 '25 вопросов',
 '41:17',
 '39:25',
 '02:00:56',
 '55:55',
 '01:58:16',
 '9 问题',
 '01:34',
 '32:58',
 '01:02:48',
 '1 domande',
 '00:26',
 '17:15',
 '43:48',
 '01:33',
 '01:18:47',
 '01:08:02',
 '01:49:00',
 '56:40',
 '01:38:08',
 '01:27:20',
 '01:30:27',
 '02:36:27',
 '01:31:22',
 '04:55',
 '01:19:29',
 '04:03:25',
 '7.4 MB',
 '13 soru',
 '02:36:41',
 '02:57:33',
 '33:12',
 '01:06:47',
 '02:06:14',
 '02:06:54',
 '08:35',
 '31:57',
 '34:21',
 '09:46',
 '31:35',
 '02:15:18',
 '46:35',
 '55:22',
 '13 questions',
 '06:01',
 '47:47',
 '01:51',
 '03:48',
 '01:52:18',
 '01:42:18',
 '17:35',
 '45:21',
 '51:34',
 '51:58',
 '41:14',
 '02:18:37',
 '10:58',
 '36:39',
 '53:01',
 '02:09:58',
 '13:01',
 '54:11',
 '01:56:17',
 '31:18',
 '02:46',
 '42:52',
 '01:01:50',
 '27:55',
 '01:06:11',
 '21:47',
 '02:25',
 '02:40:

In [51]:
udemy_lectures = []
page = None
for pk in tqdm((html_files & json_files)):
    try:
        udemy_lectures.append(get_lecture_dic(pk))
    except Exception as e:
        continue

  8%|▊         | 5038/60130 [08:07<1:28:49, 10.34it/s]


KeyboardInterrupt: 

In [50]:
udemy_cat_tag_df[udemy_cat_tag_df["id"]==page]

,id,isAccess,category,tags


In [49]:
print(udemy_cat_tag_df[udemy_cat_tag_df["id"]==page]["category"].item())
        

ValueError: can only convert an array of size 1 to a Python scalar

In [43]:
what_you_learn = list(map(lambda x:x.text, page.find_all(class_="objective--objective-item--AHk7S")))
"||".join(list(map(lambda x:x, what_you_learn)))


'How to Install the Steam Advanced Sessions plugin.||How to host an online server on Steam.||How to design & create a fully functional Server Browser.||How to filter for servers through server names, languages & more.||How to create, search and join online game servers.||How to package the project for friends and family to play together.'

In [33]:

list(map(lambda x:x.text, page.find_all(class_="objective--objective-item--AHk7S")))


['How to Install the Steam Advanced Sessions plugin.',
 'How to host an online server on Steam.',
 'How to design & create a fully functional Server Browser.',
 'How to filter for servers through server names, languages & more.',
 'How to create, search and join online game servers.',
 'How to package the project for friends and family to play together.']

In [20]:
pd.DataFrame(udemy_lectures)

,site_lecture_id,name,image,category_name,price_original,price_sale,level,summary,description_summary,site_review_rating,site_review_count,site_student_count,review_sum,review_count,total_time,curriculum,instructor,site_type,site_link
0,u4429452,Visual Basic - VB dot Net Programming,https://img-c.udemycdn.com/course/750x422/4429...,프로그래밍 언어,15000,30000,ALL,The Introduction to VB .Net||Control Propertie...,Learn Visual Basic dot Net and Windows Forms b...,4.204759,73,5477,0,0,66,"{'curriculum': [{'time': 66, 'item_count': 19,...",DIGIFLAX E-LEARNING,UDEMY,https://udemy.com/course/vb-dot-net-programming/


In [8]:
price_dic = json_description["price_text"]["data"]["pricing_result"]["price"]
sale_price_dic = json_description["price_text"]["data"]["pricing_result"]["list_price"]
int(currency2won.get(price_dic["currency"])*price_dic["amount"])//5000*5000
int(currency2won.get(sale_price_dic["currency"])*sale_price_dic["amount"])//5000*5000

30000

In [ ]:
udemy_cat_tag_df[udemy_cat_tag_df["id"]==352378]["tags"].item()
udemy_cat_tag_df[udemy_cat_tag_df["id"]==352378]["category"].item()